# Link Layer Terminology

- **nodes**: hosts and routers
- **links**: communication channels that connect adjacent nodes along communication path
    - wired links
    - wireless links
    - LANs
- **frame**: layer-2 packet, encapsulate datagram
- **data-link layer** has responsibility of transferring datagram from one node to **physically adjacent** node over a link

# Link Layer Services

- **framing, link access**
    - encapsulate datagram into frame, adding header and trailer
    - if the link is shared by multiple host, link layer controls access to the link (who transmit first)
    - channel access if shared medium
    - "MAC" (Medium Access Address) addresses used in frame headers to identify src & dest
        - different from IP address
- **delivery between adjacent nodes**
    - wireless links: high error rates
    - low bit-error link: fibre
- **flow control**
    - pacing between adjacent sending and receiving nodes
    - ensure the sender is not overloading the receiver
- **error detection**
    - errors caused by signal attenuation and noise
    - receiver detects presence of errors
        - signals sender for retransmission or drops frame
- **error correction**
    - receiver identifies and corrects bit errors without resorting to retransmission
- **half-duplex and full-duplex**
    - with half duplex, nodes at both ends of link can transmit, but not at the same time

# Link Layer Implementation

- link layer is implemented in each and every host
- link layer implemented in "adapter" (*network interface card*, NIC) or on a chip
    - Ethernet card, 802.11 card, Ethernet chipset
    - implements link, physical layer
- attaches into a host's system buses
- combination of hardware, software, firmware

<img src="img/Snip20190916_75.png" width=50%/>


# Error Detection

- **EDC**: Error Detection and Correction bits (redundancy), larger EDC field yields better detection and correction
- *D*: Data protected by error checking, may include header fields

<img src="img/Snip20190916_76.png" width=50%/>

- datagram comes from the network layer
- after error checking is passed, the datagram is passed back network layer

## Parity Checking
- *odd parity*: number of 1's must be an odd number
- *even parity*: number of 1's must be an even number

- *single bit parity*: detect single bit errors

<img src="img/Snip20190916_78.png" width=30%/>

- *two-dimensional bit parity*
    - detect and correct single bit errors
    - organize a sequence of bits as a matrix
    - compute parity bits for rows and columns
    - correctable single bit error

<img src="img/Snip20190916_79.png" width=40%/>
<img src="img/Snip20190916_80.png" width=40%/>

## Cyclic Redundancy Check

- view data bits, $D$, as a binary number
- choose $r+1$ bit pattern (generator), $G$
- goal: choose $r$ CRC bits, $R$, such that
    - $<D, R>$ exactly divisble by $G$ (modulo 2 division, not regular division)
    - receiver knows $G$, and test $<D, R>$ by dividing it by $G$
        - if the remainder is 0, there is no error
        - if the remainder is not 0, there is error
    - CRC can detect all burst errors less than $r+1$ bits
- widely used in practice (Ethernet, 802.11 WiFi)

<img src="img/Snip20190916_81.png" width=60%/>

- example: $G=(1001)_2$, $D=(101110)_2$, find R
    - we want $D*2^r \textrm{ XOR } R = nG$, equivalently
    - $D*2^r = nG \textrm{ XOR } R$ 
        - $\textrm{XOR } R$ on both sides
        - XOR is associative, so `a XOR (b XOR c) = (a XOR b) XOR c`
    - so if we modulo 2 divide $D*2^r$ by $G$, the remainder $R$ should satisfy $R=\textrm{remainder}[{D*2^r \over G}]$ 
<img src="img/Snip20190916_82.png" width=40%/>



# Multiple Access Links, Protocols

### Links
- point-to-point
- broadcast (shared wire or medium)

### Multiple Access Protocols
- distributed algorithm that determines how nodes share channel, i.e., determine when node can transmit
- communication about channel sharing must use channel itself, and no out-of-band channel for coordination
- useful when there is a single shared broadcast channel
- two or more simultaneous transmissions by nodes: interference
    - **collision** if a node receives two or more signals at the same time

#### Ideal Multiple Aceess Protocol
- *given*: broadcast channel of rate $R$ bps
- *desiderata*
    1. when one node wants to transmit, it can send at rate $R$
    2. when $M$ nodes want to transmit, each can send at average rate $R/M$
    3. fully decentralized
        - no special node to coordinate transmissions
        - no synchronization of clocks, slots
    4. simple

### MAC Protocols Taxonomy

- **channel partitioning**
    - divide channel into smaller "pieces" (time slots, frequency, code)
    - allocate piece to node for exclusive use
- **random access**
    - channel not divided, allow collisions
    - "recover" from collisions
- **taking turns**
    - nodes take turns, but nodes with more to send can take longer turns




## Channel Partitioning MAC Protocols

### Time Division Multiple Access (TDMA)
- access to channel in "rounds"
- each station gets fixed length slot (length = pkt trans time) in each round
- unused slots go idle
- **advantages**: fair, collision free
- **disadvantages**: waste of bandwidth, unnecessary waiting

*example*: 6-station LAN, 1, 3, and 4 have pkt, slots 2, 5, and 6 idle

<img src="img/Snip20190917_83.png" width=80%/>

### Frequency Division Multiple Access (FDMA)
- channel spectrum divided into frequency bands
- each station assigned fixed frequency band
- unused transmission time in frequency bands go idle
- **advantages**: fair, collision free
- **disadvantages**: waste of bandwidth, unnecessary waiting

*example*: 6-station LAN, 1, 3, and 4 have pkt, frequency bands 2, 5, and 6 idle

<img src="img/Snip20190917_84.png" width=80%/>

## Random Access Protocols

- when node has packet to send
    - transmit at full channel data rate $R$
    - no *a priori* coordination among nodes
- with two or more tansmitting nodes, there will be collision
- random access MAC protocol specifies
    - how to detect collisions
    - how to recover from collisions (e.g., via delayed retransmissions)

### Slotted ALOHA
- assumptions
    - all frames are the same size
    - time divided into equal size slots (time to transmit 1 frame)
    - nodes start to transmit only at slot beginning
    - nodes are synchronized
    - if 2 or more nodes transmit in a slot, all nodes detect collision
- operation
    - when node obtains fresh frame, transmits in next slot
        - if no collision, node can send new frame in the next slot
        - if collision, node retransmits frame in each subsequent slot with probability $p$ until success

<img src="img/Snip20190917_85.png" width=80%/>

- pros
    - single active node can continuously transmit at full rate of channel
    - highly decentralized, only slots in nodes need to be in sync
- cons
    - collisions, wasting slots
    - idle slots
    - nodes may be able to detect collision in less than time to transmit packet
    - clock synchronization
- efficiency: long-run fraction of successful slots
    - suppose $N$ nodes with many frames to send, each transmits in slot with probability $p$
    - probability that a given node has success in a slot: $p(1-p)^{N-1}$
    - probability that any node has a success: $Np(1-p)^{N-1}$
    - max efficiency: find $p^*$ that maximizes $Np(1-p)^{N-1}$, take limit of $Np^*(1-p^*)^{N-1}$ as $N\rightarrow \infty$, we get
        - $\textrm{max efficiency} = 1/e= 0.37$
    - so at best, channel used for useful transmissions 37% of time

### Pure (Unslotted) ALOHA
- simpler, no synchronization
- when frame first arrives, transmit immediately
- collision probability increasses, as frame sent at $t_0$ can collide with other frames sent in $[t_0 - 1, t_0 + 1]$

<img src="img/Snip20190917_86.png" width=80%/>

- efficiency
\begin{equation}
P(\textrm{success by given node})
\end{equation}
\begin{equation}
= P(\textrm{node transmits}) . P(\textrm{no other node transmits in [t_0 - 1, t_0]}) . P(\textrm{no other node transmits in [t_0, t_0 + 1]})
\end{equation}
\begin{equation}
= p*(1-p)^{N-1}*(1-p)^{N-1} = p*(1-p)^{2(N-1)}
\end{equation}

choosing optimum $p$ and then let $n \rightarrow \infty$, we get max efficency $1/(2e) = 0.18$ which is worse than slotted Aloha

### CSMA (Carrier Sense Multiple Access)

- **listen before transmit**
    - *if channel sensed idle*: transmit entire frame
    - *if channel sensed busy*: defer transmission
- collisions can still occur
    - propagation delay means two nodes may not hear each other's transmission
    - entire packet transmission time wasted
    - distance & propagation delay affect collision probability

### CSMA/CD (Collision Detection)

- carrier sensing, deferral as in CSMA
    - collisions are **detected** within short time
    - colliding transmissions abort, reducing channel waste
- collision detection
    - easy in wired LANs, measure signal strengths, compare transmitted, received signals
    - difficult in wireless LANs, received signal strength overwhelmed by local transmission strength

<img src="img/Snip20190920_118.png" width=60%>

### Ethernet CSMA/CD Algorithm

1. NIC receives datagram from network layer, creates a frame
2. sense channel
     a. if idle, starts frame transmission
     b. if busy, waits until channel idle, then transmits
3. if NIC transmits entire frame without detecting another transmission, NIC is done with frame
4. if NIC detects another transmission while transmitting, aborts and sends jam signal 
5. after aborting, NIC enters **binary (exponential) backoff**
    - after $m$th collision, NIC chooses $K$ at random from $\{0, 1, 2,...,2^m - 1\}$, then NIC waits for $K\times \textrm{time required to transmit 512 bits}$, return to step 2
    - longer backoff interval with more collisions

<img src="img/Snip20190920_119.png" width=80%>

- medium sensing is done for **96** bit-times
    - Interframe Gap
- jamming signal length is **48** bits, and it creates enough energy on the medium for collision detection
- $T_p$ = 512 bit-times
- $i$: collision counter, saturates at 10

### CSMA/CD Efficiency

\begin{equation}
\textrm{efficiency} = {1 \over 1 + 5t_{prop}/t_{trans}}
\end{equation}

- $t_{prop}$: maximum propagation delay between 2 nodes in LAN
- $t_{trans}$: time to transmit max-size frame

### CSMA/CA

TODO (ignore for now)


# Channel Partitioning vs Random Access

Protocol |Channel Partitioning|Random Access
--- | --- | ---
Low Traffic| inefficient, delay in channel access, idle slots | efficient, single node can fully utilize the channel 
High Traffic| all slots will be used, efficient, fair| increased number of collisions, larger overhead 

## "Taking turns" MAC Protocols

### Polling
- master node "invites" slave nodes to transmit in turn
- typically used with "dumb" slave devices
- no collisions and no empty slots
- drawbacks
    - polling overhead
    - latency
    - single point of failure

### Token Passing
- control **token** passed from one node to the next sequentially
- drawbacks
    - token overhead
    - latency
    - single point of failure (token)


# LANs: MAC addresses and ARP

- 32-bit IP address
    - *network-layer* address for interface
    - used for layer 3 (network layer) forwarding
- MAC (or LAN or physical or Ethernet) address
    - function: used "locally" to get frame from one interface to another physically-connected interface (same network, in IP-addressing sense)
    - 48 bit MAC address (for most LANs) burned in NIC ROM, also sometimes software settable
    - e.g., 1A-2F-BB-76-09-AD (hexadecimal base 16 notation)

<img src="img/Snip20190923_1.png" width=80%>

- each adapter on LAN has unique *LAN* address
- MAC address allocation administered by IEEE
- manufacturer buys portion of MAC address space to assure uniqueness
- MAC address is like Social Security Number where IP address is like postal address
- MAC flat address: portability
    - can move LAN card from one LAN to another
- IP hierachical address is not portable
    - address depends on IP subnet to which node is attached

# ARP: Address Resolution Protocol

- figure out interface's MAC address given its IP address
- **ARP Table**: each IP node (host, router) on LAN has table
    - IP/MAC address mappings for some LAN nodes:
        - `<IP address; MAC address; TTL>`
        - TTL (Time To Live): time after which address mapping will be forgotten

<img src="img/Snip20190926_16.png" width=60%/>



# ARP: Same LAN

<img src="img/Snip20190926_17.png" width=60%/>

- A wants to send datagram to B, but B's MAC address is not in A's ARP table
- A **broadcasts** ARP query packet, containing B's IP address is sent
    - dest MAC address = FF-FF-FF-FF-FF-FF
    - all nodes on LAN receive ARP query
- B receives ARP packet, replies to A with B's MAC address, frame sent to A's MAC address (unicast)
- A caches IP-to-MAC address pair in its ARP table until TTL
    - soft state: information that times out unless refreshed
- ARP is "plug-and-play", nodes create their ARP tables without intervention from net administrator

<img src="img/Snip20190926_19.png" width=40%/>

# Addressing: Routing to Another LAN

Send datagram from A to B via R: focus on addressing, at IP (datagram) and MAC layer (frame)

- assume A knows B's IP address
- assume A knows IP address of its first hop router, R
- assume A knows R's MAC address

<img src="img/Snip20190926_20.png" width=80%/>

1. A creates IP datagram with IP source A, destination B
2. A creates link-layer frame with R's MAC address as destination, frame contains A-to-B IP datagram

<img src="img/Snip20190926_21.png" width=80%/>

3. Frame sent from A to R
4. Frame received at R, datagram removed, passes up to IP

<img src="img/Snip20190926_22.png" width=80%/>

5. R forwards datagram with IP source A, destination B
6. R creates link-layer frame with B's MAC address as destination, frame contains A-to-B IP datagram

<img src="img/Snip20190926_23.png" width=80%/>

# Ethernet

- a family of computer networking technologies commonly used in local area networks (LAN)
- cheap, first widely used LAN technology
- simpler, cheaper than token LANs and ATM
- kept up with speed race: 10 Mbps - 10 Gbps

## Topology

- Bus: popular through mid 90s, all nodes in same collison domain (can collide with each other)

- Star: prevails today
    - active **switch** in center
    - each "spoke" runs a separate Ethernet protocol, nodes do not collide with each other

<img src="img/Snip20190926_24.png" width=80%/>
   
## Frame Structure

- sending adapter encapsulates IP datagram (or other network layer protocol packet) in ***Ethernet frame***

<img src="img/Snip20190926_25.png" width=80%/>

- header consists of preamble, destination address, source address, and type
- trailer is the CRC

- *data*
    - contains the IP datagram
    - maximum transmission unit (MTU) of Ethernet is 1,500 bytes
    - if the IP datagram exceeds 1,500 bytes, the host has to fragment the datagram
    - minimum size of the data field is 46 bytes, if the actual data is less than 46 bytes, it is stuffed to 46 bytes
- *preamble*
    - clock rate at the sender & the receiver should be the same
    - preamble is used to synchronize receiver, sender clock rates to mitigate drifts in transmission rates
    - use byte pattern 10101010 to wake up the receiver
    - use byte pattern 10101011 to alert that actual data is coming
    - 8 bytes
- *addresses*: 6-byte source, destination MAC address
    - if adapter receives frame with matching destination address, or with broadcast addess (e.g., ARP packet), it passes data in frame to network layer protocol
    - otherwise, adapater discards frame
- *type*: indicates higher layer protocol (mostly IP)
    - 2 bytes
- *CRC*: cyclic redundancy check at receiver
    - error detected: frame is dropped
    - 4 bytes

## Unreliable, Connectionless

- Ethernet's MAC protocol: unslotted CSMA/CD with binary backoff
- Connectionless: no handshaking between sending and receiving NICs
- Unreliable: receiving NIC doesnt send ACKs or NACKs to sending NIC
    - data in dropped frames recovered only if initial sender uses higher layer Reliable Data Transfer (e.g., TCP), otherwise dropped data is lost

## 802.3 Ethernet Standards: Link & Physical Layers

- *many* different Ethernet standards
    - common MAC protocol and frame format
    - different speeds and different physical layer media
- MAC protocol and frame format
    - e.g., 100BASE-TX
    - 100: the capacity of the link
    - BASE: the modulation type
    - TX: type of the cable


# Ethernet Switch

- objective: how to build a LAN
- Ethernet switch -> link layer switch (L2 switch)
- the swicth does not need a ARP table, as it simply forwards frames to physically connected nodes by physical address


- *link-layer device: takes an active role*
    - store and forward Ethernet frames
    - examine incoming frame's MAC address, selectively forward frame to one-or-more outgoing links when frame is to be forwarded on segment, uses CSMA/CD to access segment
- *transparent*
    - switches do not have MAC address
    - hosts are unaware of presence of switches
- *plug-and-play, self-learning*
    - switches do not need to be configured

<img src="img/Snip20190927_29.png" width=60%/>

- hosts have dedicated, direct connection to switch
- switches buffer packets
- ethernet protocol used on each incoming link, but no collisions; full duplex
    - each link is its own collision domain
- **switching**: A-to-A' and B-to-B' can transmit simultaneously without collisions

## Switch Forwarding Table

- Each switch will build its own switching table, which is the mapping between MAC addresses and switch interfaces
    - to answer questions like how to reach A' via interface 4

MAC Address|Interface|TTL
---|---|---
MAC_C|3|20 Min

- switch *learns* which hosts can be reached through which interfaces
    - when frame received, switch "learns" location of sender: incoming LAN segment
    - records sender/location pair in switch table

<img src="img/Snip20190927_30.png" width=60%/>

<img src="img/Snip20190927_31.png" width=60%/>

## Switch Frame Filtering/Forwarding

- When a frame is received at switch
    1. record incoming link, MAC address of sending host
    2. index switch table using MAC destination address
    3. find entry for destination
        1. if found, then 
            1. if destination on segment is from which frame arrived, then drop frame
            2. otherwise forward frame on interface indicated by entry
        2. if not found, flood the packet (forward on all interfaces except arriving interface)
        
- Destination location is unknown: *flood*
- Destination location is known: *selectively send on just one link*

## Interconnected Switches

<img src="img/Snip20190927_32.png" width=60%/>

**Example** Sending from A to G

1. $S_1$ receives frame from A, record A -> 1, flood
2. all devices connected to $S_1$ except for $S_4$ drops frame, $S_4$ received frame from MAC $A$ at its interface 1, record that, then flood
3. all devices connected to $S_4$ except for $S_3$ drops frame, $S_3$ received frame from MAC $A$ at its interface 1, record that, then flood
4. G receives



# Institutional Network

<img src="img/Snip20190930_42.png" width=80%/>

# Switches vs Routers

- both are **store-and-forward**
    - *routers*
        - network-layer devices (examine network-layer headers)
        - pros: hierachical addressing, hence traffic isolation - src to dest, routing
        - cons: large processing time, needs IP configuration
    - *switches*
        - link-layer devices (examine link-layer headers)
        - pros: fast, plug-and-play
        - cons: broadcast storm, large ARP table for large network

- both have **forwarding tables**
    - *routers*: compute tables using routing algorithms, IP addresses
    - *switches*: learn forwarding table using flooding, learning, MAC addresses
        
<img src="img/Snip20190930_43.png" width=60%/>

# Data Center Networks

- challenges
    - multiple applications, each serving massive numbers of clients
    - managing/balancing load, avoiding processing, networking, data bottlenecks

## Load Balancer: Application-Layer Routing

- receives external client requests
- directs workload within data center
- returns results to external client (hiding data center internals from client)

<img src="img/Snip20190930_44.png" width=80%/>

## Interconnection

- rich interconnection among switches, racks
    - increased throughput between racks (multiple routing paths possible)
    - increased reliability via redundancy
    - use spanning tree to avoid broadcast storm

<img src="img/Snip20190930_45.png" width=80%/>

# Wireless LAN (WLAN, WiFi)

<img src="img/Snip20190930_46.png" width=80%/>

- basic building block in a WiFi is called the Basic Service Set (BSS)
- a BSS will consist of one or more wireless stations and one Access Point (AP)
- each BSS has an identity (BSSID)
- the Access Point (AP) continuously broadcast the BSSID and its MAC address
- a BSS is usually connected to the Internet using wired Ethernet (802.3) link to a router
- a group of BSS's can form an Extended BSS and they have the same identity/BSSID
    - {BSS} connected with a Distributed System
- a group of wireless stations can form a network to communicate among each other without an AP, this type of network is called ad-hoc network
    - Independent BSS (IBSS) = BSS - AP

<img src="img/Snip20190930_47.png" width=70%/>

- the AP will implement 802.3 (Ethernet) link layer as well as 802.11 (WiFi) link layer
    - the AP can convert a WiFi frame into an Ethernet frame and vice versa

## Operation Modes of WiFi MAC protocol

- DCF (Distributed Coordination Function)
- PCF (Point Coordination Function)

<img src="img/Snip20190930_48.png" width=70%/>


### PCF (Optional Mode)

- leads to waste of bandwidth if a scheduled node has no traffic
- based on the idea of polling ("taking-turns" MAC)
- The AP
    - operates as the **central controller** in the BSS
    - decides who transmits and when
    - *no contention* for medium access
    - can follow a round-robin policy to allocate slots

### DCF Mode (Required Mode)

- all nodes, including the AP, compete for medium access
- the AP does not operate as a central controller
- the AP
    - need not be used
        - computers can directly communicate among themselves (Ad Hoc)
        - used to *provide connectivity to the Internet*

<img src="img/Snip20190930_49.png" width=80%/>

<img src="img/Snip20190930_50.png" width=80%/>

<img src="img/Snip20190930_51.png" width=80%/>

<img src="img/Snip20190930_53.png" width=80%/>


## Problems in WLAN

### Hidden Terminal Problem

<img src="img/Snip20190930_54.png" width=60%/>

- the problem is due to C being hidden(far away) from A
- collision occurs at B, but A cannot detect it
- solution: CSMA/CA (Collision Avoidance) 

### Exposed Terminal Problem

<img src="img/Snip20190930_55.png" width=30%/>

- Problem: loss of opportunity to transmit, loss of bandwidth

- A is transmitting a frame to D
- B knows that someone is transmitting, if B transmits a frame to C, it does not collide with A's at D
- however, B does not transmit because B is unaware of D's location
- the problem is due to B being exposed to A's Tx

## WLAN MAC: CSMA/CA

### Frame Types

#### RTS/CTS

<img src="img/Snip20190930_59.png" width=70%/>

- sender of RTS/CST says: "I want to use the medium for this amount of duration"
- Frame Control: Frame type
- FCS: Frame Check Sequence (CRC)
- RA: Receiver's MAC address
- TA: Transmitter's MAC address

#### DATA

<img src="img/Snip20191001_68.png" width=70%/>

- Header: starting from Frame control to A4
    - A1 ~ A4: MAC addresses (usage depends on the context)
    - Sequence Control: numbering of packets
- Trailer: FCS
- Both header and trailer are considered overhead: 34 bytes

### WLAN: 1-hop communication between client and the first router

<img src="img/Snip20191001_69.png" width=70%/>

- SA: Source Address (the origin of the frame)
- TA: Transmitter Address (who is transmittting)
- RA: Receiver Address (who is receiving)
- DA: Destination Address (who is the intended receiver 1-hop away)

#### Case 1: Client-to-router

- WiFi-Eth. Converter creates a new Ethernet frame from the WiFi frame

**WiFi Frame**

$A_1$|$A_2$|$A_3$|$A_4$
---|---|---|---
RA|SA|DA|Not Used

- $A_1$ = RA (BSSID) = MAC addr. of AP
- $A_2$ = SA/TA
- $A_3$ = DA
- $A_4$ = Not Used

**Converted Ethernet Frame**

.|Dest. Addr.|Src. Addr.|...
---|---|---|---
Preamble|DA from $A_3$|SA from $A_2$|...


#### Case 2: Router-to-client

- WiFi-Eth. Converter creates a new WiFi frame from the Ethernet frame

**Ethernet Frame**

.|Dest. Addr.|Src. Addr.|...
---|---|---|---
Preamble|DA|SA|...

**Converted WiFi Frame**

$A_1$|$A_2$|$A_3$|$A_4$
---|---|---|---
DA|BSSID|SA|Not Used

- $A_1$ = DA from dest. addr. of Ethernet Frame
- $A_2$ = BSSID
- $A_3$ = SA from src. addr. of Ethernet Frame
- $A_4$ = Not Used

#### Case 3: Client-to-router via 2 AP

<img src="img/Snip20191001_71.png" width=70%/>

- Frame 1: Ethernet frame from client MAC_C to MAC_AP1

.|Dest. Addr.|Src. Addr.|...
---|---|---|---
Preamble|DA (MAC_R)|SA (MAC_C)|...

- Frame 2: WiFi frame from MAC_AP1 to MAC_AP2

$A_1$|$A_2$|$A_3$|$A_4$
---|---|---|---
RA (MAC_AP2)|TA (MAC_AP1)|DA (MAC_R)|SA (MAC_C)

- Frame 3: Ethernet frame from MAC_AP2 to MAC_R

.|Dest. Addr.|Src. Addr.|...
---|---|---|---
Preamble|DA (MAC_R)|SA (MAC_C)|...


# WLAN MAC: CSMA/CA

## Collision Avoidance

- **PHY-level carrier sensing**: done in receiver hardware
- **Virtual carrier sensing** (a MAC protocol concept):
    - an integer variable in each node, called **NAV**(*Network Allocation Vector*), which indicates whether or not a *nearby hidden* node is *likely* to be transmitting now
        - NAV > 0: most likely another node is transmitting
        - NAV = 0: no one nearby is transmitting
        - ~~NAV < 0: not exist~~
- Transmit condition: Medium is idle at the receiver (carrier is absent AND NAV = 0)


## Nav in Detail

- recall the handshake mechanism with RTS and CTS
    - a **duration** in $\mu s$ field in frame header indicates the length of time the sender of the frame may use the medium
    - **all nodes** which receive RTS and CTS (and DATA) update their NAV as follows

```
initially, NAV = 0
With each passing µs,
    if (NAV > 0) then NAV = NAV - 1
    else stop decrementing NAV
if a frame (e.g., RTS, CTS, DATA) with duration field is received
    NAV = Max(NAV, duration)
```

## Timing Intervals

- the IEEE 802.11 MAC defines 4 timing intervals
    - 2 at the PHY level
        - *SIFS*: Short Inter-Frame Space, waiting time before transmitting any two consecutive frames (of any type, e.g., RTS, CTS, DATA, ACK), 10 µs
        - *aSlot*: 20 µs, propagation delay + some hardware delay, chosen such that a node can determine if another node initiated a Tx aSlot time before
    - 2 at the MAC level
        - *PIFS*: Priority (in PCF) IFS (=*SIFS+aSlot*)
        - *DIFS*: Distributed IFS (=*PIFS+aSlot*)

- timing intervals are used to control priority
- SIFS < PIFS < DIFS
    - by keeping SIFS shortest, it is ensured that an ongoing cycle of $T_x$ (with or without handshake) is not disturbed
    - no one else can $T_x$ between RTS, CTS, DATA, ACK
    - PIFS < DIFS enables an AP to become the controller of a BSS

<img src="img/IMG_0196.jpg" width=50%/>

## DCF/ Handshake using RTS/CST (DATA: A$\rightarrow$B)

- duration in RTS should be sufficient for the sender to finish the transmission cycle, i.e., until the sender receives an ACK

<img src="img/Snip20191004_87.png" width=80%/>

## DCF with Handshake: Transmitter

<img src="img/Snip20191004_88.png" width=70%/>

## DCF with Handshake: Receiver

<img src="img/Snip20191004_89.png" width=70%/>

## DCF without Handshake

<img src="img/Snip20191004_90.png" width=70%/>

- RTS/CTS frames are not exchanged
- the idea of NAV still applies
    - *all nodes* process the received RTS/CTS of others
    - so even in no-handshake mode, a node still listen to RTS and CTS and update its NAV accordingly

# PCF 

- AP alternates between PCF and DCF modes
- AP operates as the controller 
    - if AP finds medium to be **idle** (no carrier and NAV = 0) for PIFS, it transmits a **beacon** frame
    - beacon contains a **CFPMaxDuration** field (CFP: Contention Free Period)
    - nodes receiving a beacon update their NAV to **CFPMaxDuration**
    - nodes perceive the medium to be busy for **CFPMaxDuration**
    - nodes do not transmit unless asked to do so by the AP
    - after **transmitting a beacon**, AP waits for PIFS before transmitting one of the following frames
        - DATA
        - CF Poll (Contention Free Poll)
        - DATA + CF Poll
        - ACK
        - CF End

**CF Poll Frame**

<img src="img/Snip20191007_118.png" width=80%/>

- if ACK is not received, user 1 waits until it is polled again
- if a user does not have data, it sends a null DATA frame

**DATA + CF Poll Frame**

<img src="img/Snip20191007_120.png" width=50%/>

- if AP does not receive an ACK, it retransmits data after PIFS
- if user 1 does not receive ACK, it does not retransmit data

**Unicast and broadcast**

<img src="img/Snip20191007_121.png" width=80%/>

**CF End Frame**

- receiving nodes set NAV = 0
- identifies the end of CF period



# WiFi

- WiFi operates in the frequency band 2.4 Ghz with bandwidth 85 MHz
- the bandwidth is divided into 14 channels
- the WiFi channels have a BW of 22 MHz and they are separated by 5 Mhz
- WiFi has *overlapping* channels
- any two channels are said to be non-overlapping if they are separated by four or more channels


<img src="img/Snip20191007_122.png" width=80%/>